In [ ]:
%cd /content/drive/MyDrive/CS410

/content/drive/MyDrive/CS410


In [ ]:
!pip install celluloid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import math
import pandas as pd
from IPython import display
from time import sleep
import math
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from celluloid import Camera

In [ ]:
def Sphere(ind):
    sum = 0
    for i in ind:
        sum += i**2
    return sum

In [ ]:
def Zakharov(x):
  d = len(x)
  sum1 = 0.0
  sum2 = 0.0
  for i in range(d):
    sum1 += x[i]**2
    sum2 += 0.5 * (i + 1) * x[i]
  y = sum1 + sum2**2 + sum2**4
  return y

In [ ]:
def Rosenbrock(ind):
    sum = 0
    for i in range(len(ind) - 1):
        sum += 100 * (ind[i + 1] - ind[i]**2)**2 + (ind[i] - 1)**2
    return sum 

In [ ]:
def Michalewicz(x):
  d = len(x)
  sum = 0.0
  for i in range(d):
    sum += np.sin(x[i])*pow((np.sin((i+1)*pow(x[i],2)/np.pi)),20) 
  return sum

In [ ]:
def Ackley(d):
    a = 20
    b = 0.2
    c = 2 * np.pi
    sum1 = 0
    sum2 = 0
    for i in range(len(d)):
        sum1 += d[i] ** 2
        sum2 += np.cos(c * d[i])
    term1 = -a * np.exp(-b * np.sqrt(sum1 / len(d)))
    term2 = -np.exp(sum2 / len(d))

    return term1 + term2 + a + np.exp(1)

In [ ]:
def  CEM(test_function, dimensions, bounds, popsize, num_elite, sigma_init, extra_std, seed_number, max_evals):
    np.random.seed(seed_number)
    eps = 1e-4
    bound_lower, bound_upper = np.asarray(bounds).T
    sigma = sigma_init * np.eye(dimensions)

    diff = np.fabs(bound_lower - bound_upper)
    n_evals = 0
    num_evals = []
    mu = np.random.rand(dimensions) - (bound_upper + 1)
    generation_count = 0
    all_mu = []
    all_sigma = []
    all_offspring = []
    all_pops = []
    all_sigma = []
    all_elite = []
    all_fitness = []
    while True:
    # for i in range(10000):
        if n_evals > max_evals: 
            break
        all_mu.append(mu)
        all_sigma.append(sigma)

        x = np.random.multivariate_normal(mu, sigma, popsize)
        all_pops.append(np.copy(x))
        if np.sum(x) > 1e150 or np.sum(x) < -1e150:
            break
        all_offspring.append(x)
        fitness = np.array([test_function(x[i]) for i in range(popsize)])
        n_evals += popsize
        best_fitness = max(fitness) 
        all_fitness.append(best_fitness)
        if best_fitness < eps:
            break

        elite_idx = fitness.argsort()[:num_elite]
        all_elite.append(elite_idx)

        sigma = np.zeros_like(sigma)

        for i in range(num_elite):
            z = x[elite_idx[i]] - mu
            z = z.reshape(-1, 1)
            # sigma += tf.matmul(z.T, z)
            sigma += (z.T @ z)

        sigma += np.eye(dimensions)*extra_std
        
        all_sigma.append(sigma)
        sigma *= (1/num_elite)
        mu = np.mean(x[elite_idx], axis=0)

        generation_count += 1
        num_evals.append(n_evals)

    all_mu.append(mu)
    best_results = mu.copy()
    best_fitness = test_function(mu)
    return all_pops, all_sigma, all_mu, all_fitness, num_evals, generation_count

In [ ]:
seed_number = 20521394

bound_lower, bound_upper = -3, 3
popsize = 32
dimension = 2
max_evals = 1e5
F_scale=0.8
num_elite = 10
sigma_init = 4
cross_prob=0.7
extra_std = 0.01

In [ ]:
def  CrossEntropyMethod(test_function, dimensions, bounds, popsize, num_elite, sigma_init, extra_std, seed_number, max_evals):
    np.random.rand(seed_number)
    # all_pops = []
    generation_count = 0
    results = []
    cov_matrix = []
    
    max_evals = 5000 if popsize < 512 else 10000
    
    results, cov_matrix, ind, fitness, evals, generation_count =  CEM(test_function, dimensions, bounds, popsize, num_elite, sigma_init, extra_std, seed_number, max_evals)
    # print(results)
    # print(cov_matrix)
    bound_lower = -30
    bound_upper = 30
    x = np.linspace(bound_lower, bound_upper, 100)
    y = np.linspace(bound_lower, bound_upper, 100)
    X, Y = np.meshgrid(x, y)
    Z = test_function([X, Y])
    
    fig = plt.figure(figsize=(12, 12))
    camera = Camera(fig)
    plt.contourf(X, Y, Z, popsize, cmap='viridis')
    plt.axis('square')
    plt.scatter(1, 1, marker='*')
    
    for generation in range(len(results)):
        plt.contourf(X, Y, Z, popsize, cmap='viridis')
        plt.axis('square')
        plt.scatter(1, 1, marker='*')
        plt.scatter(results[generation][:, 0], results[generation][:, 1], c='#ff0000', marker='o')
        plt.scatter(cov_matrix[generation][:, 0], cov_matrix[generation][:, 1], c='#ff1493', marker='x')
        plt.plot()
        plt.pause(0.1)
        camera.snap()
    plt.show()
    anim = camera.animate()
    
    #save the animation as a gif file
    anim.save('Sphere' + "-CEM-" + str(popsize) + ".gif",writer="pillow")

In [ ]:
all_fitness = []
num_evaluation = []
    
test_function = Sphere
seed_number = 20521394
    
popsize_array = [32, 64, 128, 256, 512, 1024]
for popsize in popsize_array:
    print(f"Popsize = {popsize}")
    if dimension == 2:
        max_evals = 1e5
        bounds = [(bound_lower, bound_upper)]*dimension
        mean_result = np.mean(all_fitness)
        stddev_result = np.std(all_fitness)
        mean_evaluation = np.mean(num_evaluation)
            
         CrossEntropyMethod(test_function, dimension, bounds, popsize, num_elite, sigma_init, extra_std, seed_number, max_evals) 

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def  CrossEntropyMethod(test_function, dimensions, bounds, popsize, num_elite, sigma_init, extra_std, seed_number, max_evals):
    np.random.rand(seed_number)
    # all_pops = []
    generation_count = 0
    results = []
    cov_matrix = []
    
    max_evals = 5000 if popsize < 512 else 10000
    
    results, cov_matrix, ind, fitness, evals, generation_count =  CEM(test_function, dimensions, bounds, popsize, num_elite, sigma_init, extra_std, seed_number, max_evals)
    # print(results)
    # print(cov_matrix)
    bound_lower = -30
    bound_upper = 30
    x = np.linspace(bound_lower, bound_upper, 100)
    y = np.linspace(bound_lower, bound_upper, 100)
    X, Y = np.meshgrid(x, y)
    Z = test_function([X, Y])
    
    fig = plt.figure(figsize=(12, 12))
    camera = Camera(fig)
    plt.contourf(X, Y, Z, popsize, cmap='viridis')
    plt.axis('square')
    plt.scatter(1, 1, marker='*')
    
    for generation in range(len(results)):
        plt.contourf(X, Y, Z, popsize, cmap='viridis')
        plt.axis('square')
        plt.scatter(1, 1, marker='*')
        plt.scatter(results[generation][:, 0], results[generation][:, 1], c='#ff0000', marker='o')
        plt.scatter(cov_matrix[generation][:, 0], cov_matrix[generation][:, 1], c='#ff1493', marker='x')
        plt.plot()
        plt.pause(0.1)
        camera.snap()
    plt.show()
    anim = camera.animate()
    
    #save the animation as a gif file
    anim.save('Sphere' + "-CEM-" + str(popsize) + ".gif",writer="pillow")

In [ ]:
all_fitness = []
num_evaluation = []
    
test_function = Sphere
seed_number = 20521394
    
popsize_array = [32, 64, 128, 256, 512, 1024]
for popsize in popsize_array:
    print(f"Popsize = {popsize}")
    if dimension == 2:
        max_evals = 1e5
        bounds = [(bound_lower, bound_upper)]*dimension
        mean_result = np.mean(all_fitness)
        stddev_result = np.std(all_fitness)
        mean_evaluation = np.mean(num_evaluation)
            
         CrossEntropyMethod(test_function, dimension, bounds, popsize, num_elite, sigma_init, extra_std, seed_number, max_evals) 

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def  CrossEntropyMethod(test_function, dimensions, bounds, popsize, num_elite, sigma_init, extra_std, seed_number, max_evals):
    np.random.rand(seed_number)
    # all_pops = []
    generation_count = 0
    results = []
    cov_matrix = []
    
    max_evals = 5000 if popsize < 512 else 10000
    
    results, cov_matrix, ind, fitness, evals, generation_count =  CEM(test_function, dimensions, bounds, popsize, num_elite, sigma_init, extra_std, seed_number, max_evals)
    # print(results)
    # print(cov_matrix)
    bound_lower = -30
    bound_upper = 30
    x = np.linspace(bound_lower, bound_upper, 100)
    y = np.linspace(bound_lower, bound_upper, 100)
    X, Y = np.meshgrid(x, y)
    Z = test_function([X, Y])
    
    fig = plt.figure(figsize=(12, 12))
    camera = Camera(fig)
    plt.contourf(X, Y, Z, popsize, cmap='viridis')
    plt.axis('square')
    plt.scatter(1, 1, marker='*')
    
    for generation in range(len(results)):
        plt.contourf(X, Y, Z, popsize, cmap='viridis')
        plt.axis('square')
        plt.scatter(1, 1, marker='*')
        plt.scatter(results[generation][:, 0], results[generation][:, 1], c='#ff0000', marker='o')
        plt.scatter(cov_matrix[generation][:, 0], cov_matrix[generation][:, 1], c='#ff1493', marker='x')
        plt.plot()
        plt.pause(0.1)
        camera.snap()
    plt.show()
    anim = camera.animate()
    
    #save the animation as a gif file
    anim.save('Zakharov' + "-CEM-" + str(popsize) + ".gif",writer="pillow")

In [ ]:
all_fitness = []
num_evaluation = []
    
test_function = Zakharov
seed_number = 20521394
    
popsize_array = [32, 64, 128, 256, 512, 1024]
for popsize in popsize_array:
    print(f"Popsize = {popsize}")
    if dimension == 2:
        max_evals = 1e5
        bounds = [(bound_lower, bound_upper)]*dimension
        mean_result = np.mean(all_fitness)
        stddev_result = np.std(all_fitness)
        mean_evaluation = np.mean(num_evaluation)
            
         CrossEntropyMethod(test_function, dimension, bounds, popsize, num_elite, sigma_init, extra_std, seed_number, max_evals) 

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def  CrossEntropyMethod(test_function, dimensions, bounds, popsize, num_elite, sigma_init, extra_std, seed_number, max_evals):
    np.random.rand(seed_number)
    # all_pops = []
    generation_count = 0
    results = []
    cov_matrix = []
    
    max_evals = 5000 if popsize < 512 else 10000
    
    results, cov_matrix, ind, fitness, evals, generation_count =  CEM(test_function, dimensions, bounds, popsize, num_elite, sigma_init, extra_std, seed_number, max_evals)
    # print(results)
    # print(cov_matrix)
    bound_lower = -30
    bound_upper = 30
    x = np.linspace(bound_lower, bound_upper, 100)
    y = np.linspace(bound_lower, bound_upper, 100)
    X, Y = np.meshgrid(x, y)
    Z = test_function([X, Y])
    
    fig = plt.figure(figsize=(12, 12))
    camera = Camera(fig)
    plt.contourf(X, Y, Z, popsize, cmap='viridis')
    plt.axis('square')
    plt.scatter(1, 1, marker='*')
    
    for generation in range(len(results)):
        plt.contourf(X, Y, Z, popsize, cmap='viridis')
        plt.axis('square')
        plt.scatter(1, 1, marker='*')
        plt.scatter(results[generation][:, 0], results[generation][:, 1], c='#ff0000', marker='o')
        plt.scatter(cov_matrix[generation][:, 0], cov_matrix[generation][:, 1], c='#ff1493', marker='x')
        plt.plot()
        plt.pause(0.1)
        camera.snap()
    plt.show()
    anim = camera.animate()
    
    #save the animation as a gif file
    anim.save('Rosenbrock' + "-CEM-" + str(popsize) + ".gif",writer="pillow")

In [ ]:
all_fitness = []
num_evaluation = []
    
test_function = Rosenbrock
seed_number = 20521394
    
popsize_array = [32, 64, 128, 256, 512, 1024]
for popsize in popsize_array:
    print(f"Popsize = {popsize}")
    if dimension == 2:
        max_evals = 1e5
        bounds = [(bound_lower, bound_upper)]*dimension
        mean_result = np.mean(all_fitness)
        stddev_result = np.std(all_fitness)
        mean_evaluation = np.mean(num_evaluation)
            
         CrossEntropyMethod(test_function, dimension, bounds, popsize, num_elite, sigma_init, extra_std, seed_number, max_evals) 

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def  CrossEntropyMethod(test_function, dimensions, bounds, popsize, num_elite, sigma_init, extra_std, seed_number, max_evals):
    np.random.rand(seed_number)
    # all_pops = []
    generation_count = 0
    results = []
    cov_matrix = []
    
    max_evals = 5000 if popsize < 512 else 10000
    
    results, cov_matrix, ind, fitness, evals, generation_count =  CEM(test_function, dimensions, bounds, popsize, num_elite, sigma_init, extra_std, seed_number, max_evals)
    # print(results)
    # print(cov_matrix)
    bound_lower = -30
    bound_upper = 30
    x = np.linspace(bound_lower, bound_upper, 100)
    y = np.linspace(bound_lower, bound_upper, 100)
    X, Y = np.meshgrid(x, y)
    Z = test_function([X, Y])
    
    fig = plt.figure(figsize=(12, 12))
    camera = Camera(fig)
    plt.contourf(X, Y, Z, popsize, cmap='viridis')
    plt.axis('square')
    plt.scatter(1, 1, marker='*')
    
    for generation in range(len(results)):
        plt.contourf(X, Y, Z, popsize, cmap='viridis')
        plt.axis('square')
        plt.scatter(1, 1, marker='*')
        plt.scatter(results[generation][:, 0], results[generation][:, 1], c='#ff0000', marker='o')
        plt.scatter(cov_matrix[generation][:, 0], cov_matrix[generation][:, 1], c='#ff1493', marker='x')
        plt.plot()
        plt.pause(0.1)
        camera.snap()
    plt.show()
    anim = camera.animate()
    
    #save the animation as a gif file
    anim.save('Michalewicz' + "-CEM-" + str(popsize) + ".gif",writer="pillow")

In [ ]:
all_fitness = []
num_evaluation = []
    
test_function = Michalewicz
seed_number = 20521394
    
popsize_array = [32, 64, 128, 256, 512, 1024]
for popsize in popsize_array:
    print(f"Popsize = {popsize}")
    if dimension == 2:
        max_evals = 1e5
        bounds = [(bound_lower, bound_upper)]*dimension
        mean_result = np.mean(all_fitness)
        stddev_result = np.std(all_fitness)
        mean_evaluation = np.mean(num_evaluation)
            
         CrossEntropyMethod(test_function, dimension, bounds, popsize, num_elite, sigma_init, extra_std, seed_number, max_evals) 

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def  CrossEntropyMethod(test_function, dimensions, bounds, popsize, num_elite, sigma_init, extra_std, seed_number, max_evals):
    np.random.rand(seed_number)
    # all_pops = []
    generation_count = 0
    results = []
    cov_matrix = []
    
    max_evals = 5000 if popsize < 512 else 10000
    
    results, cov_matrix, ind, fitness, evals, generation_count =  CEM(test_function, dimensions, bounds, popsize, num_elite, sigma_init, extra_std, seed_number, max_evals)
    # print(results)
    # print(cov_matrix)
    bound_lower = -30
    bound_upper = 30
    x = np.linspace(bound_lower, bound_upper, 100)
    y = np.linspace(bound_lower, bound_upper, 100)
    X, Y = np.meshgrid(x, y)
    Z = test_function([X, Y])
    
    fig = plt.figure(figsize=(12, 12))
    camera = Camera(fig)
    plt.contourf(X, Y, Z, popsize, cmap='viridis')
    plt.axis('square')
    plt.scatter(1, 1, marker='*')
    
    for generation in range(len(results)):
        plt.contourf(X, Y, Z, popsize, cmap='viridis')
        plt.axis('square')
        plt.scatter(1, 1, marker='*')
        plt.scatter(results[generation][:, 0], results[generation][:, 1], c='#ff0000', marker='o')
        plt.scatter(cov_matrix[generation][:, 0], cov_matrix[generation][:, 1], c='#ff1493', marker='x')
        plt.plot()
        plt.pause(0.1)
        camera.snap()
    plt.show()
    anim = camera.animate()
    
    #save the animation as a gif file
    anim.save('Ackley' + "-CEM-" + str(popsize) + ".gif",writer="pillow")

In [ ]:
all_fitness = []
num_evaluation = []
    
test_function = Ackley
seed_number = 20521394
    
popsize_array = [32, 64, 128, 256, 512, 1024]
for popsize in popsize_array:
    print(f"Popsize = {popsize}")
    if dimension == 2:
        max_evals = 1e5
        bounds = [(bound_lower, bound_upper)]*dimension
        mean_result = np.mean(all_fitness)
        stddev_result = np.std(all_fitness)
        mean_evaluation = np.mean(num_evaluation)
            
         CrossEntropyMethod(test_function, dimension, bounds, popsize, num_elite, sigma_init, extra_std, seed_number, max_evals) 

Output hidden; open in https://colab.research.google.com to view.